In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
import os
import sys

In [2]:
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

In [3]:
!pip install import-ipynb

You should consider upgrading via the 'D:\Project\ai\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import import_ipynb

import ab
import data_preprocessing

In [ ]:
def predict_next_hour(input_csv_path):
    
    BASE_DIR = os.getcwd()
    MODEL_PATH = os.path.join(BASE_DIR, 'models/trained_model.h5')
    SCALER_PATH = os.path.join(BASE_DIR, 'models/scaler.pkl')
    
    print(f"Looking for model at: {MODEL_PATH}")
    if not os.path.exists(MODEL_PATH) or not os.path.exists(SCALER_PATH):
        print("Model or Scaler not found.") 
        return

    print("model and scaler")
    
    
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    
    scaler = joblib.load(SCALER_PATH)

    
    print(f"Reading data from {input_csv_path}...")
    df = pd.read_csv(input_csv_path)
    
    
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    
    
    df = ab.feature_engineering_pipeline(df)
    
    
    cols_to_drop = ['date', 'day_of_week', 'cluster', 'outlier_flag', 'rv1', 'rv2']
    features = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

    
    print("Scaling data")
    scaled_array = scaler.transform(features)
    
    
    WINDOW_SIZE = 24
    if len(scaled_array) < WINDOW_SIZE:
        print(f"Not enough data! Need at least {WINDOW_SIZE} rows.")
        return

    
    last_sequence = scaled_array[-WINDOW_SIZE:] 
    input_batch = last_sequence.reshape((1, WINDOW_SIZE, last_sequence.shape[1]))

    
    print("Predicting")
    
    prediction_scaled = model.predict(input_batch)
    pred_value_scaled = prediction_scaled[0][0]

    
    target_col_name = 'Appliances'
    try:
        target_idx = list(features.columns).index(target_col_name)
    except ValueError:
        print("Target column not found.")
        return

    
    data_min = scaler.data_min_[target_idx]
    data_range = scaler.data_range_[target_idx]
    
    predicted_energy = (pred_value_scaled * data_range) + data_min

    print("\n" + "="*40)
    print(f"PREDICTION RESULT")
    print(f"Predicted Consumption: {predicted_energy:.2f} Wh")
    print("="*40 + "\n")
    
    return predicted_energy

test_file = os.path.join(os.getcwd(), '../data/raw/energy_data_set.csv')

prediction = predict_next_hour(test_file)

Looking for model at: d:\Project\src\models/trained_model.h5
model and scaler
Reading data from d:\Project\src\../data/raw/energy_data_set.csv...
Scaling data...


<string>:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


Predicting
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step

PREDICTION RESULT
Predicted Consumption: 296.44 Wh

